In [1]:
# As usual, a bit of setup
import time
import numpy as np
import matplotlib.pyplot as plt
from boxx import *
from cs231n.classifiers.fc_net import *
from cs231n.data_utils import get_CIFAR10_data
from cs231n.gradient_check import eval_numerical_gradient, eval_numerical_gradient_array
from cs231n.solver import Solver

%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

# for auto-reloading external modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

In [2]:
np.random.seed(231)
N, D = 4, 5
x = 5 * np.random.randn(N, D) + 12 # (N, D)
gamma = np.random.randn(D)         # (D,)
beta = np.random.randn(D)          # (D,)
dout = np.random.randn(N, D)       # (N, D)

bn_param = {'mode': 'train'}

_, cache = batchnorm_forward(x, gamma, beta, bn_param)
dx, dgamma, dbeta = batchnorm_backward(dout, cache)

In [18]:
x, x_normalized, gamma, beta, mu, var, eps = cache
    
dbeta = np.sum(1. * dout, axis=0)                                    # (D,)
dgamma_x_normalized = 1. * dout                                      # (N, D)
dgamma = np.sum(x_normalized * dgamma_x_normalized, axis=0)          # (D,)
dx_normalized = gamma * dgamma_x_normalized                          # (N, D)
dx_minus_mu_route_1 = 1. / np.sqrt(var + eps) * dx_normalized        # (N, D)
d1_over_sqrt_var_plus_eps = np.sum((x - mu) * dx_normalized, axis=0) #(D,)
dsqrt_var_plus_eps = -1. / (var + eps) * d1_over_sqrt_var_plus_eps   #(D,)
dvar = 0.5 * 1. /np.sqrt(var + eps) * dsqrt_var_plus_eps             # (D,)
dx_minus_mu_squared = 1. / N * np.ones((N, D)) * dvar                # (D,) -> (N, D)
dx_minus_mu_route_2 = 2. * (x - mu) * dx_minus_mu_squared            # (N, D)
dx_minus_mu = dx_minus_mu_route_1 + dx_minus_mu_route_2              # (N, D)
dmu = -1. * np.sum(dx_minus_mu, axis=0)                              # (D,)
dx_route_1 = 1. * dmu                                                # (N, D)
dx_route_2 = 1. / N * np.ones((N, D)) * dmu                          # (D,) -> (N, D)
dx = dx_route_1 + dx_route_2                                         # (N, D)

# Shapes
```
x            (N, D)
x_normalized (N, D)
gamma        (D,)
beta         (D,)
mu           (D,)
sigma        (D,)
eps           1

dbeta        (D,)
dgamma_x_normalized (N, D)
dgamma       (D,)
dx_normalized(N, D)
```

In [16]:
print(dx_minus_mu.shape)

(4, 5)


In [32]:
(np.random.rand(*x.shape) < p) / p

array([[0., 0., 1., 1., 1.],
       [1., 1., 1., 1., 0.],
       [1., 0., 0., 1., 0.],
       [0., 1., 1., 1., 1.]])

In [22]:
p = 0.5